In [1]:
import json
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import urllib
from decouple import config

FDA_KEY = config('FDA_KEY')

In [2]:
url='https://api.nal.usda.gov/fdc/v1/foods/search?api_key=DEMO_KEY&query=Chocolate'
json_url = urllib.request.urlopen(url)
results = json.loads(json_url.read())

In [3]:
results
total_pages = results['totalPages']

In [17]:
flattened_data = pd.json_normalize(results)
flattened_data1 = pd.json_normalize(flattened_data.foods[0])
#flattened_data1

In [5]:
nutrients = flattened_data1['foodNutrients']
nutrients[0][0]

{'nutrientId': 1003,
 'nutrientName': 'Protein',
 'nutrientNumber': '203',
 'unitName': 'G',
 'derivationCode': 'LCCS',
 'derivationDescription': 'Calculated from value per serving size measure',
 'value': 5.71}

In [6]:
full_nutrient_list = [(nutrient_item['nutrientId'], nutrient_item['nutrientName']) for nutrient in nutrients for nutrient_item in nutrient]
nutrient_list = dict(set(full_nutrient_list)) #removes duplicates

In [ ]:
len(nutrient_list)

In [ ]:
nutrient_list

In [7]:
def get_sugar_details (sugar_id, nutrient_list):
    #returns dictionary containing sugar value
    for nutrient in nutrient_list:
        if nutrient['nutrientId'] == sugar_id:
            return nutrient['value']
    return np.nan


In [8]:
#extract nutrient_ids_related to sugar
sugar_nutrient_ids = dict(filter(lambda elem: 'sugar' in elem[1].lower(), nutrient_list.items()))
#add sugar columns
for sugar_id in sugar_nutrient_ids.items():
    flattened_data1[sugar_id[1]] = [get_sugar_details(sugar_id[0],nutrient_list) for nutrient_list in flattened_data1['foodNutrients']]




In [9]:
#rename sugar columns
flattened_data1 = flattened_data1.rename(columns = {'Sugars, total including NLEA':'Total_Sugars', 'Sugars, added':'Added_Sugars'})

#identify chocolate with the least amount of sugar
sorted_chocolate = flattened_data1.sort_values(by=['Total_Sugars'])


In [10]:
#write to file
output_df = sorted_chocolate[['fdcId','brandOwner','ingredients','Total_Sugars','Added_Sugars']]
output_df.to_csv('chocolate.csv',index = False)

In [11]:
df=pd.DataFrame()
#run loop to extract all chocolates listed in the USDA list
for i in range(1,total_pages):
    url=f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={FDA_KEY}&query=Chocolate&pageNumber={i}'
    json_url = urllib.request.urlopen(url)
    results = json.loads(json_url.read())
    flattened_data = pd.json_normalize(results)
    flattened_data = pd.json_normalize(results)
    flattened_data1 = pd.json_normalize(flattened_data.foods[0])
    df = df.append(flattened_data1)

HTTPError: HTTP Error 400: 

In [18]:
#even with 400 error still retrieved 10,000 items
#df

In [13]:
for sugar_id in sugar_nutrient_ids.items():
    df[sugar_id[1]] = [get_sugar_details(sugar_id[0],nutrient_list) for nutrient_list in df['foodNutrients']]


In [14]:
#rename sugar columns
df = df.rename(columns = {'Sugars, total including NLEA':'Total_Sugars', 'Sugars, added':'Added_Sugars'})

#sort by Total Sugars descending
sorted_chocolate = df.sort_values(by=['Total_Sugars'])


In [16]:
#remove chocolate that doesn't have a brandOwner
sorted_chocolate.dropna(subset=['brandOwner'], inplace=True)

#write to file
output_df = sorted_chocolate[['fdcId','brandOwner','ingredients','Total_Sugars','Added_Sugars']].head(1000)
output_df.to_csv('chocolate_1k.csv',index = False)